# PyTorch
PyTorch je framework pro strojové učení založený na knihovně Torch, používaný pro aplikace umělé inteligence, například v počítačovém vidění či zpracování přirozené mluvy. Původně byl vyvinutý Meta AI, nyní je součástí Linux Foundation. Je to bezplatný a open-source software vydaný pod upravenou licencí BSD.

Je optimalizovaný pro práci s tenzory a grafy. Umožňuje provádět výpočty na GPU.
    
Základní vlastnosti:
1. PyTorch tensor (velice podobný Numpy ndarray).
1. Přímé, necyklické grafy (Direct acyclic graphs) pro modelování modelů umělé inteligence a hlubokého učení.
1. Dynamicky sestavované grafy (např. Tenforflow pracuje se statickými grafy) je možné měnit vstupy i operace za běhu.
1. Pythoní knihovna, pro "pythonistu" jednoduchá na pochopení a použití (vs Keras, Tensorflow).

Instalace pomocí Anaconda

```conda install pytorch torchvision cpuonly -c pytorch```

Instalace pomocí pip

```pip3 install torch torchvision```

In [ ]:
!pip3 install numpy matplotlib torch

In [ ]:
import torch
import numpy as np
# Základní vlastnosti tenzoru
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

### Operace s Tenzory

In [ ]:
# Standartní slicing jako v numpy
tensor = torch.ones(4, 4)
tensor[:,1] = 0
print(tensor)

In [ ]:
# Přesun tensoru na GPU, pokud je k dispozici
if torch.cuda.is_available():
  tensor = tensor.to('cuda')
print(f"Device tensor is stored on: {tensor.device}")

In [ ]:
# Tranzpozice tenzoru
t_inverse = tensor.T 
print(tensor)
print(t_inverse)

In [ ]:
# Sčítání tenzorů
t0 = tensor + tensor
print(t0)
t1 = tensor + 5
print(t1)
t2 = tensor.add(5)
print(t2)

In [ ]:
# Násobení tenzorů jednotlivé prvky
t3 = tensor * tensor
print(t3)
t4 = tensor.mul(tensor)
print(t4)

In [ ]:
# Maticové násobení 
t5 = tensor @ tensor.T
print(t5)
t6 = tensor.matmul(tensor.T)
print(t6)

In [ ]:
# Spojování tenzorů
t7 = torch.cat([tensor, tensor, tensor], dim=1)
print(t7)

In [ ]:
# Suma tenzoru
t8 = tensor.sum()
print(t8)

In [ ]:
# Maximum a minimum
tmax = tensor.max()
print(tmax)
tmin = tensor.min()
print(tmin)

In [ ]:
# In-place operace
print(tensor)
tensor.add_(5)
print(tensor)

### Most do NumPy (Bringe with NumPy)
Torch Tensor na CPU a NumPy Array  mohou sdílet paměť -> změna jednoho změní i druhý.

#### PyTorch Tenzor do NumPy Array

In [ ]:
t = torch.ones(5)
print(f"t: {t}, type: {type(t)}")
n = t.numpy()
print(f"n: {n}, type: {type(n)}")
# změna tenzoru se odrazí i v NumPy array
t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

#### NumPy array do PyTorch Tenzoru

In [ ]:
n = np.ones(5)
print(f"n: {n}")
t = torch.from_numpy(n)
print(f"t: {t}")
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

# Trénovaní neuronové sítě
Trénování neuronové sítě probíhá ve dvou stále se opakujících krocích:

**Forward Propagation:** Neuronová síť vrací svůj odhad správného výstupu. Data prochází jednotlivými vrstvami/funkcemi modelu a je vypočítána výstupní hodnota.

**Backward Propagation:** Během backpropagation, model upravuje své parametry podle chyby na výstupu. Toto je prováděno pomocí postupného návratu od výstupu ke vstupu, sledují se změny gradientu a parametry funkcí modelu jsou optimalizovány pomocí gradient descent. Pro vizualizaci backpropagation doporučuji třeba [video od 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

## Autograd - jednoduchá ukázka
### Analytické řešení v Numpy

In [ ]:
!pip3 install matplotlib

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import math

a = np.linspace(0., 2. * math.pi, 25)
print(a)

In [ ]:
b = np.sin(a)
plt.plot(a, b)

In [ ]:
# ukazka vypoctu
c = 2 * b
d = c + 1
out = d.sum()
out

Zde je analytické řešení pro náš případ:

1. b = np.sin(a) - derivace sin(a) je cos(a).
1. c = 2 * b - derivace 2 * sin(a) je 2 * cos(a), protože derivace konstanty je konstanta a derivace sin(a) je cos(a).
1. d = c + 1 - derivace 2 * cos(a) + 1 je stále 2 * cos(a), protože derivace konstanty je 0.
1. out = d.sum() - zde se výstupy z předchozího kroku sumarizují, ale protože chceme gradient každého prvku a vzhledem k out, musíme se vrátit k tomu, že gradient každého prvku a vzhledem k jeho příspěvku v out je 2 * cos(a).

In [ ]:
# vypocet gradientu
gradient_a = 2 * np.cos(a)
gradient_a

In [ ]:
plt.plot(a, gradient_a)

### PyTorch backward()

In [ ]:
import torch

a = torch.linspace(0., 2. * math.pi, steps=25, requires_grad=True)
print(a)

In [ ]:
b = torch.sin(a)
plt.plot(a.detach(), b.detach())

In [ ]:
# ukazka vypoctu
c = 2 * b
d = c + 1
out = d.sum()
out

In [ ]:
# autograd
out.backward()
print(a.grad)
plt.plot(a.detach(), a.grad.detach())

## Gradient descent
Gradient Descent (Gradientní sestup) je optimalizační algoritmus používaný v strojovém učení k minimalizaci chyby výstupu modelu (en: cost function), která kvantifikuje, jak "daleko" je model od ideálního řešení. Základní myšlenka gradient descent spočívá v iterativním upravování parametrů modelu (například vah v neuronové síti) s cílem postupně snížit chybu modelu. Výpočet probíhá iterativně:

1. Výpočet Gradientu:
Gradient je vektor, který udává směr nejstrmějšího stoupání funkce. V kontextu optimalizace chceme najít opačný směr, tedy směr nejstrmějšího klesání, aby se minimalizovala funkce nákladů.
Pro každý parametr modelu (např. váhu) se vypočítá parciální derivace cost function vzhledem k tomuto parametru, což indikuje, jak malá změna v parametru ovlivní celkovou hodnotu cost function.
1. Aktualizace Parametrů  
Parametry modelu se aktualizují podle vzorce:  
$$ \theta_{n+1} = \theta_n - \eta \cdot \nabla_\theta J(\theta)$$  
kde:  
$\theta$ jsou parametry modelu, které se mají optimalizovat.  
$\eta$ je rychlost učení (learning rate), což je kladný skalár určující velikost kroku při aktualizaci parametrů.  
$\nabla_\theta $ je gradient funkce nákladů $J$ vzhledem k parametrům $\theta$, který udává směr nejstrmějšího stoupání funkce nákladů.  
Při aktualizaci se pohybujeme v opačném směru k nalezení minima funkce nákladů.

## Perceptron in Pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definice modelu
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.linear = nn.Linear(2, 1)  # 2 vstupy -> 1 výstup
        self.aktivace = nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.aktivace(x)
        return x

In [ ]:
# Inicializace modelu, ztrátové funkce a optimalizátoru
model = Perceptron()
criterion = nn.BCELoss() # Binární cross-entropy ztráta pro klasifikační úlohy
SDGoptimizer = optim.SGD(model.parameters(), lr=0.1) # Stochastic gradient descent optimizer

# Trénovací data
X = torch.randn(5, 2, requires_grad=False)
Y = torch.randn(5, 1, requires_grad=False)
X[0]

In [ ]:
# Vahy perceptronu
print(model.linear.weight)
print(model.linear.weight.grad)

In [ ]:
# vypocet vystupu modelu
prediction = model(X[0])
print(prediction)
Y[0]

In [ ]:
# vzorova loss function
loss = (Y[0] - prediction).pow(2).sum()
print(loss)

In [ ]:
# vypocet gradientu
loss.backward()
print(model.linear.weight)
print(model.linear.weight.grad)

In [ ]:
# uprava vah pomoci SDG
SDGoptimizer.step()
print(model.linear.weight)
print(model.linear.weight.grad)

In [ ]:
# kumulace gradientu
for i in range(5):
    prediction = model(X[i])
    loss = (Y[i] - prediction).pow(2).sum()
    loss.backward()

print(model.linear.weight)
print(model.linear.weight.grad)

In [ ]:
# vynulovani gradientu
SDGoptimizer.zero_grad(set_to_none=False)
print(model.linear.weight)
print(model.linear.weight.grad)

Na konci se může vrátit na start s upravenými váhami.

LOSS = BCE (Binary Cross Entropy)

## Entropy
Entropy je míra nejistoty nebo překvapení. V teorii informací je entropie pravděpodobnostní míra, která měří množství informace obsažené v pravděpodobnostním rozdělení. V kontextu strojového učení je entropie často používána jako ztrátová funkce pro klasifikační modely.
výpočet informační entropie:
čím výšší pravděpodobnost, tím nižší entropie.
Příklad:
pravděpodobnost výhry Čr je 99%
pravděpodobnost výhry USA je 1%
-99% * log2(99%) - 1% * log2(1%) = 0.08

Využijeme sigmoid aktivační fci.
y = our -> O or 1 -> y = k
y^ = predicted -> 0 or 1 -> y^ = p

B(k|p) = p^k * (1-p)^(1-k) -> if p -> k = 1; if 1-p -> k = 0
P(yi|y^i) = y^i * (1 - y^i)^(1 - y^i) -> BCE

P = suma ale s násobením (P(yi|y^i))
výpočet -> pi^yi * (1 - pi)^(1 - yi) -> BCE (p = y^)

Moc náročné na paměť cheme SUM

P = sum

## Binary Cross Entropy
- to předtím bylo useless

BCE = -1/n * sum(yi * log(y^i) + (1 - yi) * log(1 - y^i))

## Gradient Descent
- chyba = loss
- lr = learning rate
- W = váhy
- dL/dW = derivace chyby podle vah
- dL = derivace chyby
- dWold = derivace původní váhy
- Wnew = nová váha
- Wold = původní váha

Wnew = Wold - lr * dL/dWold


## NonBinary
- používá aktivační fci softmax
- 

